### import de bibliotecas

In [1]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')

C:\Anaconda\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sandr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### extração dos tweets da base de dados do Aos Fatos

In [2]:
import pandas as pd
#df = pd.read_excel (r'Bases_VisagioHackathon\Base_tweets_saúde_Aos_Fatos.xlsx')
xls = pd.ExcelFile('Bases_VisagioHackathon\Base_tweets_saúde_Aos_Fatos.xlsx')
df1 = pd.read_excel(xls, 'mar-abr')
data1 = df1[['id','tweet']].copy()
df2 = pd.read_excel(xls, 'abr-mai')
data2 = df2[['id','tweet']].copy()
df3 = pd.read_excel(xls, 'mai-jun')
data3 = df3[['id','tweet']].copy()
df4 = pd.read_excel(xls, 'jun-jul')
data4 = df4[['id','tweet']].copy()
df5 = pd.read_excel(xls, 'jul-ago')
data5 = df5[['id','tweet']].copy()
df6 = pd.read_excel(xls, 'ago-set')
data6 = df6[['id','tweet']].copy()

tabs = [data1, data2, data3, data4, data5, data6]
data = pd.concat(tabs)

### tratamento dos tweets recebidos
1) retirada de pontuações e "stop words" das frases <br>
2) separação de cada uma das palavras do tweet

In [3]:
stopwords = nltk.corpus.stopwords.words('portuguese')

#print(len(data))

data_filtrada = pd.DataFrame(columns = ['id','tweet'])
#print(data_filtrada)

for i in range(len(data)):
    frase_original = data.iloc[i,1].replace(',', ' ').replace('.', ' ').replace(':', ' ').replace('-', ' ').lower()
    #print(frase_original)
    frase_filtrada = ''
    for palavra in frase_original.split():
        if palavra not in stopwords:
            frase_filtrada += (palavra  + ' ')
    #print(frase_filtrada)
    data_filtrada = data_filtrada.append({'id': data.iloc[i,0], 'tweet': frase_filtrada}, ignore_index=True)
    
#print(word_tokenize(data_filtrada.iloc[0,1]))
len_data = len(data_filtrada)

#tagged_data = [TaggedDocument(words=(word_tokenize(data_filtrada.iloc[i,1])), tags=[str(i)]) for i in range(len_data)]
new_tagged_data = [(word_tokenize(data_filtrada.iloc[i,1])) for i in range(len_data)]

### criação do modelo referente ao word embedding em Língua Portuguesa disponibilizado pelo NILC
http://nilc.icmc.usp.br/nilc/index.php/repositorio-de-word-embeddings-do-nilc

In [4]:
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format('cbow_s50.txt')

In [5]:
model_saude = Word2Vec(new_tagged_data, size=50, window=10, min_count=2, workers=10)
model_saude.train(new_tagged_data,total_examples=len(new_tagged_data),epochs=100)

(22050961, 27472500)

### tópicos relacionados à entrada de busca feita pelo usuário
- são analisados possíveis tópicos gerados a partir de dois modelos: o de tweets (com mais termos voltados à saúde), e o do NILC

In [6]:
def find_topics(phrase, model_saude, model):
    topics = []
    #print(phrase)
    phrase = phrase.replace(',', ' ').replace('.', ' ').replace(':', ' ').replace('-', ' ').lower()
    #print(phrase)
    frase_filtrada = ''
    
    for palavra in phrase.split():
        if palavra not in stopwords:
            frase_filtrada += (palavra  + ' ')
    new_phrase = (word_tokenize(frase_filtrada))
    #print(new_phrase)
    
    try:
        topics.append(model.wv.most_similar(positive=new_phrase, topn=3))
        #print(topics)
    except KeyError:
        print("sem vocabulario correspondente")
        
    try:
        topics.append(model_saude.wv.most_similar(positive=new_phrase, topn=3))
    except KeyError:
        print("sem vocabulario em saude correspondente")
    
    return topics

In [7]:
print(find_topics("covid", model_saude, model))

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  from ipykernel import kernelapp as app


sem vocabulario correspondente
[[('doença', 0.5649859309196472), ('todos', 0.4938358664512634), ('hidroxicloroquina', 0.4792617857456207)]]
